In [20]:
import re
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
from transformers import pipeline

In [41]:
from sentence_transformers import util

In [42]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [43]:
# api_key = os.getenv("OPENAI_API_KEY")
# if not api_key:
#     raise RuntimeError("OPENAI_API_KEY not set in environment")

In [44]:
# embeddings = OpenAIEmbeddings()
# llm = OpenAI(temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))


In [45]:
career_data = [
    {"career": "Software Developer", "keywords": ["coding", "programming", "technology", "computers"], "strengths": ["math", "computer science"]},
    {"career": "Graphic Designer", "keywords": ["art", "design", "creativity", "visual"], "strengths": ["art", "design"]},
    {"career": "Sports Coach", "keywords": ["sports", "coaching", "teamwork", "leadership"], "strengths": ["physical education", "psychology"]},
    {"career": "Environmental Scientist", "keywords": ["science", "nature", "experiments", "environment"], "strengths": ["science", "biology"]},
    {"career": "Writer", "keywords": ["writing", "stories", "creativity", "literature"], "strengths": ["english", "literature"]}
]

In [46]:
career_texts = [" ".join(c["keywords"]) for c in career_data]
career_embeddings = model.encode(career_texts, convert_to_tensor=True)

In [48]:
# vector stoer
# career_texts = [" ".join(career["keywords"]) for career in career_data]
# vector_store = FAISS.from_texts(career_texts, embeddings)

In [49]:
INTEREST_PROMPT = "Extract the main interests from the following response: {response}"
STRENGTH_PROMPT = "Extract the academic subjects the user is strong in from the following response: {response}"
EXPLANATION_PROMPT = "Explain why a career as a {career} might be a good fit for someone interested in {interests} and strong in {strengths}."

In [50]:
FALLBACK_INTEREST = "Can you tell me more about what you enjoy? For example, do you like solving problems, creating art, or maybe working with people?"
FALLBACK_STRENGTH = "Are there any subjects where you consistently do well or find interesting?"

In [51]:
def extract_info(prompt, response):
    return [word.strip().lower() for word in response.split(",") if word.strip()]

def mock_llm_response(career, interests, strengths):
    return f"A career as a {career} fits your interests in {', '.join(interests)} and strengths in {', '.join(strengths)}."

In [52]:
# def extract_info(prompt, response):
#     chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt))
#     result = chain.run(response)
#     return result.split(", ") if result else []

#     career_scores = {}
#     for doc in interest_matches:
#         career_index = career_texts.index(doc.page_content)
#         career = career_data[career_index]
#         score = 0
#         score += len(set(career["keywords"]) & set(interests))
#         score += len(set(career["strengths"]) & set(strengths))
#         career_scores[career["career"]] = score
    

#     top_careers = sorted(career_scores.items(), key=lambda x: x[1], reverse=True)[:3]
#     # return [career_data[career_texts.index(" ".join(next(c["keywords"] for c in career_data if c["career"] == career)))] for career, _ in top_careers]
#     return [word.strip() for word in response.split(",")]

In [53]:
# def get_recommendations(interests, strengths):
#     interest_query = " ".join(interests)
#     interest_matches = vector_store.similarity_search(interest_query, k=5)

#     career_scores = {}
#     for doc in interest_matches:
#         career_index = career_texts.index(doc.page_content)
#         career = career_data[career_index]
#         score = 0
#         # Interest match
#         score += len(set(career["keywords"]) & set(interests))
#         # Strength match
#         score += len(set(career["strengths"]) & set(strengths))
#         career_scores[career["career"]] = score
    
#     top_careers = sorted(career_scores.items(), key=lambda x: x[1], reverse=True)[:3]
#     return [career_data[career_texts.index(" ".join(next(c["keywords"] for c in career_data if c["career"] == career)))] for career, _ in top_careers]

In [54]:
def get_recommendations(interests, strengths):
    query = " ".join(interests)
    query_embedding = model.encode(query, convert_to_tensor=True)

    similarities = util.pytorch_cos_sim(query_embedding, career_embeddings)[0]
    top_k = similarities.topk(3)

    top_careers = []
    for idx in top_k.indices:
        career = career_data[idx]
        top_careers.append(career)

    return top_careers

In [55]:
# conv agent

In [58]:
def career_advice_agent():
    print("Hi! I'm here to help you find career paths that match your interests and strengths. Let's get started!")
    
    interest_response = input("What are some activities, subjects, or hobbies that you really enjoy? ")
    interests = extract_info(INTEREST_PROMPT, interest_response)
    if not interests or len(interests) == 1 and not interests[0]:
        print(FALLBACK_INTEREST)
        interest_response = input()
        interests = extract_info(INTEREST_PROMPT, interest_response)
    
    strength_response = input("Great! Now, what subjects do you feel you excel in or enjoy the most at school? ")
    strengths = extract_info(STRENGTH_PROMPT, strength_response)
    if not strengths or len(strengths) == 1 and not strengths[0]:
        print(FALLBACK_STRENGTH)
        strength_response = input()
        strengths = extract_info(STRENGTH_PROMPT, strength_response)
    
    recommended_careers = get_recommendations(interests, strengths)
    # print("\nBased on what you've shared, here are three career paths that might be perfect for you:")
    # for career in recommended_careers:
    #     explanation = llm(EXPLANATION_PROMPT.format(career=career["career"], interests=", ".join(interests), strengths=", ".join(strengths)))
    #     print(f"- **{career['career']}**: {explanation}")
    print("\n Based on what you shared, here are some career paths you might enjoy:\n")
    for rec in recommended_careers:
        explanation = mock_llm_response(rec["career"], interests, strengths)
        print(f" {rec['career']}: {explanation}")

    print("\nWould you like to explore more paths?")

    
    print("\nWould you like to learn more about any of these careers, or perhaps explore other options?")


In [59]:
if __name__ == "__main__":
    career_advice_agent()

Hi! I'm here to help you find career paths that match your interests and strengths. Let's get started!


What are some activities, subjects, or hobbies that you really enjoy?  playing cricket, reading, coding
Great! Now, what subjects do you feel you excel in or enjoy the most at school?  cs, maths, physics



🎯 Based on what you shared, here are some career paths you might enjoy:

🔹 Software Developer: A career as a Software Developer fits your interests in playing cricket, reading, coding and strengths in cs, maths, physics.
🔹 Writer: A career as a Writer fits your interests in playing cricket, reading, coding and strengths in cs, maths, physics.
🔹 Sports Coach: A career as a Sports Coach fits your interests in playing cricket, reading, coding and strengths in cs, maths, physics.

Would you like to explore more paths?

Would you like to learn more about any of these careers, or perhaps explore other options?
